In [1]:
# Libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import random
from scipy.stats import norm
from scipy import log,exp,sqrt,stats

In [2]:
s0= 100             # Stock price today
x= 100              # Strike price
barrier = 150       # Barrier level
T= 1                # Maturity in years
r=0.08              # Risk-free rate
sigma=0.3           # Annualized volatility
n_simulation = 51   # number of simulations

#Firm specific information
v0=200 # assumption
sigma_firm=0.25
debt=175
recovery_rate=0.25
correlation=0.2

In [3]:
# Define black and scholes call price
def bs_call (S,X,T,rfr,sigma):
    """
       Returns: Call value under Black-Schole-Merton option model
       Format   : bs_call(S0,K,rfr,T,sigma)
               S: current stock price
               X: exercise price/strike price
               T: maturity date in years
              rfr: risk-free rate (continusouly compounded)
           sigma: volatiity of underlying security 
    """ 
    d1=(log(S/X)+(rfr+sigma**2/2)*T)/(sigma*sqrt(T))
    d2 = d1-sigma*sqrt(T)
    return S*stats.norm.cdf(d1)-X*exp(-rfr*T)*stats.norm.cdf(d2)

In [4]:
def terminal_value(s0,rfr,sigma,Z,T):
    """Generates the share price at time T given some random normal values, Z"""
    return s0*np.exp((rfr-(sigma**2)/2)*T+sigma*np.sqrt(T)*Z)

In [5]:
def up_and_out_call(s0,x,T,r,sigma,n_simulation,barrier):
    """
        Returns: Call value of an up-and-out barrier option with European call
    """
    np.random.seed(123)
    n_steps = 12 # Define number of steps.
    dt = T/n_steps
    total=0
    call_est=[None]*50
    sT_est=[None]*50
    for j in range(1,n_simulation):
        sT=s0
        out=False
        for i in range(0,int(n_steps)):
            Z= sp.random.normal(size=j*1000)
            sT=sT*sp.exp((r-0.5*sigma**2)*dt+sigma*Z*sp.sqrt(dt))
            sT_est[j-1]=np.mean(sT)
            if sT_est[j-1]>barrier:
                out=True
        
        if out==False:
            total=total+bs_call(s0,x,T,r,sigma)
            call_est=total/n_simulation
    return call_est

In [6]:
def up_and_out_call_cva(s0,x,T,r,sigma,n_simulation,barrier,cva,correlation):
    """
        Returns: Call value of an up-and-out barrier option with European call adjusted for CVA
    """
    np.random.seed(123)
    n_steps = 12 # Define number of steps.
    dt = T/n_steps
    total=0
    call_est=[None]*50
    sT_est=[None]*50
    for j in range(1,n_simulation):
        sT=s0
        out=False
        for i in range(0,int(n_steps)):
            corr_matrix=np.array([[1,correlation],[correlation,1]])
            norm_matrix=sp.random.normal(size=np.array([2,50000]))
            corr_norm_matrix=np.matmul(np.linalg.cholesky(corr_matrix),norm_matrix)
            sT=sT*sp.exp((r-0.5*sigma**2)*dt+sigma*corr_norm_matrix[0,]*sp.sqrt(dt))
            sT_est[j-1]=np.mean(sT)
            if sT_est[j-1]>barrier:
                out=True
        
        if out==False:
            total=total+bs_call(s0,x,T,r,sigma)-cva
            call_est=total/n_simulation
    return call_est

In [7]:
call_price = up_and_out_call(s0,x,T,r,sigma,n_simulation,barrier)
print('Price for the Up-and-out Call without correlation and default free is = ', round(call_price,3))

Price for the Up-and-out Call without correlation and default free is =  15.403


In [8]:
corr_matrix=np.array([[1,correlation],[correlation,1]])
norm_matrix=sp.random.normal(size=np.array([2,50000]))
corr_norm_matrix=np.matmul(np.linalg.cholesky(corr_matrix),norm_matrix)
cva_estimates=[None]*50
cva_std=[None]*50

for k in range(1,n_simulation):
    term_firm_value=terminal_value(v0,r,sigma_firm,corr_norm_matrix[1,],T)
    amount_lost=sp.exp(-r*T)*(1-recovery_rate)*(term_firm_value<debt)*call_price
    cva_estimates_corr=np.mean(amount_lost)
    
d1=(log(v0/debt)+(r+sigma_firm**2/2)*T)/(sigma_firm*sqrt(T))
d2 = d1-sigma_firm*sqrt(T)
default_prob=norm.cdf(-d2)
uncorr_cva=(1-recovery_rate)*default_prob*call_price

In [9]:
call_price_with_cva = up_and_out_call_cva(s0,x,T,r,sigma,n_simulation,barrier,cva_estimates_corr,correlation)
print('Price for the Up-and-out Call with correlation and CVA impact is = ', round(call_price_with_cva,3))

Price for the Up-and-out Call with correlation and CVA impact is =  12.966
